# Import libraries

In [ ]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import pandas as pd
import numpy as np
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, BatchNormalization, Dropout, Input
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
import keras
from datetime import datetime
from sklearn import preprocessing
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import plot_model
from sklearn.model_selection import GridSearchCV
from sklearn.utils import shuffle
import pickle

# Load processed dataset
The data we load here are already shuffled and normalized.

In [ ]:
data_directory = 'dataset/3. normalized/'
x_train = pickle.load(open(data_directory + "x_train.p", "rb"))
y_train = pickle.load(open(data_directory + "y_train.p", "rb"))
x_test = pickle.load(open(data_directory + "x_test.p", "rb"))
y_test = pickle.load(open(data_directory + "y_test.p", "rb"))
print('x_train: ' + str(x_train.shape))
print('y_train: ' + str(y_train.shape))
print('x_test: ' + str(x_test.shape))
print('y_test: ' + str(y_test.shape))

# Tunable parameters

In [ ]:
epochs = 2000
batch_size = 20
learning_rate = 0.0001
decay = 0.0001

# Autoencoder #1

In [ ]:
date = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
root_directory = 'results/results-autoencoder-01/'
model_directory = root_directory + "model/"
tensorboard_directory = root_directory + "tensorboard/"
def create_model(layers, dropout):
    global date
    x = Input(shape=(x_train.shape[1],))

    y = Dense(units = layers['layer_1'], activation='relu')(x)
    y = Dropout(dropout)(y)

    if (layers['layer_2'] > 0):
        y = Dense(units = layers['layer_2'], activation='relu')(y)
        y = Dropout(dropout)(y)
    
    y = Dense(units = x_train.shape[1], activation='sigmoid')(y)
    model = Model(x, y)
    
    # Create directory
    directory = model_directory + date + '/'
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    # Write model hyper-parameters
    df = pd.DataFrame({
        'Layer 1': layers['layer_1'],
        'Layer 2': layers['layer_2'],
        'Dropout': dropout,
        'Learning Rate': learning_rate,
        'decay': decay
    }, index=[0])
    df.to_csv(model_directory + date + '/params.csv')
    
    # Write model summary
    file2 = open(directory + "summary.txt", "a")
    model.summary(print_fn=lambda line: file2.write(line + '\n'))
    file2.close()

    # Write model diagram
    plot_model(model, to_file=directory + 'model.png', show_shapes=True, show_layer_names=False)
    
    # Compile the model
    model.compile(optimizer=keras.optimizers.Adam(lr=learning_rate, decay=decay), loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])
    
    return model

# Prepare callbacks

In [ ]:
class KerasClassifierTensorBoard(KerasClassifier):
    def fit(self, x, y, **kwargs):
        global date
        date = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
        tensorboard_callback = keras.callbacks.TensorBoard(log_dir=tensorboard_directory + date + '/')
        csv_logger = keras.callbacks.CSVLogger(model_directory + date + '/epochs.csv')
        model_checkpoint = keras.callbacks.ModelCheckpoint(model_directory + date + '/weights.hdf5', save_best_only=True)
        callbacks = [tensorboard_callback, csv_logger, model_checkpoint]
        return super(KerasClassifierTensorBoard, self).fit(x, y, callbacks=callbacks, **kwargs)

# Grid search

In [ ]:
model = KerasClassifierTensorBoard(build_fn=create_model, epochs=epochs, batch_size=batch_size)

layers = [
    {'layer_1': 10, 'layer_2': 0},
    {'layer_1': 10, 'layer_2': 8},
    {'layer_1': 9, 'layer_2': 0},
    {'layer_1': 9, 'layer_2': 6},
    {'layer_1': 8, 'layer_2': 0},
    {'layer_1': 8, 'layer_2': 4}
]
dropouts = [0.5, 0.4]
param_grid = dict(
    layers=layers, 
    dropout = dropouts
)

grid = GridSearchCV(estimator = model, param_grid = param_grid, error_score=0, verbose=2, cv=[(slice(None), slice(None))], n_jobs=1, fit_params=dict(validation_data=(x_test, x_test)))
grid_result = grid.fit(x_train, x_train)